## Урок 4. СУБД MongoDB

1) Модифицировать приложение из предыдущего домашнего задания:
- приложение должно собирать вакансии сразу с двух сайтов hh.ru и superjob.ru 
- собранные данные должны быть приведены к общей структуре
- разделить зарплату на две составляющие (мин. и макс.) и сохранить в виде int. Если валюта указана другая, привести все к рублям. 

In [6]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
import requests
import re
import time

def get_link(main_link, add_link='', key_word='', page=''):
    link = main_link + add_link + key_word + page
    return link

def get_html(link, headers = {}):
    if link[0:4] == 'http':
        html = requests.get(link, headers=headers).text
    else:
        with open(link, 'rb') as f:
            html = f.read()
    return html

def parsing_html(link, headers = {}):
    html =  get_html(link, headers=headers)
    parsed_html = bs(html, 'html.parser')
    return parsed_html

def parsing_html_pages(link, pages, headers = {}):
    parsed_html_pages = bs('', 'html.parser')
    for i in pages:
        time.sleep(5)
        parsed_html_pages.append(parsing_html(link+str(i), headers=headers))
    return parsed_html_pages

def find_class(html, class_):
    found_class = html.findAll(class_=class_)
    return found_class

def find_item(html, class_, href=False):
    found_item = html.find(class_=class_, href=href)
    return found_item

def get_vacancy(html, class_):
    found_class = find_class(html, class_[0])
    vacancy_info = []
    for i in found_class:
        class_dict = {}
        vacancy = find_item(i, class_[1], href=True)
        class_dict['vacancy'] = vacancy.get_text()
        salary = find_item(i, class_[2])
        salary = salary.get_text() if salary else salary
        salary = re.findall('([\d]+)[\s]*([\d]+)', salary) if salary else ['0']
        class_dict['salary'] = [int(''.join(i)) for i in salary] if len(salary) else [0]
        link = vacancy['href']
        link = link if link[0:4]=='http' else main_link + link
        link = re.findall('http[s]?:\/\/[^\?]*', link)
        class_dict['link'] = link[0] if len(link) else ''
        vacancy_info.append(class_dict)
    return vacancy_info


main_link = 'https://hh.ru'
add_link = '/search/vacancy'
key_word = '?area=1&text=Программист Разработчик'
page = '&page='
pages = range(0, 10)
class_ = ['vacancy-serp-item', 'bloko-link HH-LinkModifier', 'vacancy-serp-item__compensation']
headers = {'User-Agent': UserAgent().chrome}
link = get_link(main_link, add_link, key_word, page)
parsed_html_pages = parsing_html_pages(link, pages, headers)
vacancy_hh = get_vacancy(parsed_html_pages, class_)
pprint(vacancy_hh)


main_link = 'https://superjob.ru'
add_link = '/vacancy/search/'
key_word = '?keywords=Программист|Разработчик'
page = '&page='
pages = range(1, 11)
class_ = ['_3syPg', '_1QIBo', '_2Wp8I']
link = get_link(main_link, add_link, key_word, page)
parsed_html_pages = parsing_html_pages(link, pages)
vacancy_sj = get_vacancy(parsed_html_pages, class_)
pprint(vacancy_sj)


[{'link': 'https://hh.ru/vacancy/32116925',
  'salary': [80000, 150000],
  'vacancy': 'Программист Javascript'},
 {'link': 'https://hh.ru/vacancy/32543357',
  'salary': [80000, 120000],
  'vacancy': 'Программист С++ (графика)'},
 {'link': 'https://hh.ru/vacancy/31399856',
  'salary': [80000, 120000],
  'vacancy': 'Программист iOS (Swift)'},
 {'link': 'https://hh.ru/vacancy/33189536',
  'salary': [55000, 70000],
  'vacancy': 'Junior PHP-разработчик'},
 {'link': 'https://hh.ru/vacancy/32901208',
  'salary': [0],
  'vacancy': 'Frontend-разработчик'},
 {'link': 'https://hh.ru/vacancy/33192429',
  'salary': [180000],
  'vacancy': 'Ведущий программист 1C / Ведущий разработчик 1С'},
 {'link': 'https://hh.ru/vacancy/29832222',
  'salary': [150000, 190000],
  'vacancy': 'C# back-end разработчик'},
 {'link': 'https://hh.ru/vacancy/32518280',
  'salary': [130000, 160000],
  'vacancy': 'Frontend разработчик'},
 {'link': 'https://hh.ru/vacancy/31273029',
  'salary': [0],
  'vacancy': 'Программист J

[{'link': 'https://superjob.ru/vakansii/web-programmist-32266222.html',
  'salary': [0],
  'vacancy': 'Web-программист, Web-разработчик'},
 {'link': 'https://superjob.ru/vakansii/programmist-32418578.html',
  'salary': [140000],
  'vacancy': 'Программист / Разработчик 1С 8.3 УПП'},
 {'link': 'https://superjob.ru/vakansii/programmist-32274027.html',
  'salary': [80000],
  'vacancy': 'Программист / Разработчик'},
 {'link': 'https://superjob.ru/vakansii/veb-programmist-32441272.html',
  'salary': [100000],
  'vacancy': 'Веб-программист, веб-разработчик'},
 {'link': 'https://superjob.ru/vakansii/programmist-razrabotchik-32408682.html',
  'salary': [0],
  'vacancy': 'Программист-разработчик (.NET) на ELMA BPM'},
 {'link': 'https://superjob.ru/vakansii/programmist-32221501.html',
  'salary': [80000],
  'vacancy': 'Программист (разработчик 1С)'},
 {'link': 'https://superjob.ru/vakansii/programmist-razrabotchik-32382559.html',
  'salary': [60000],
  'vacancy': 'Программист-разработчик'},
 {'li

 {'link': 'https://superjob.ru/vakansii/programmist-1s-32431672.html',
  'salary': [0],
  'vacancy': 'Программист 1С'},
 {'link': 'https://superjob.ru/vakansii/programmist-c-32401530.html',
  'salary': [0],
  'vacancy': 'Программист C++'},
 {'link': 'https://superjob.ru/vakansii/programmist-1s-32122929.html',
  'salary': [0],
  'vacancy': 'Программист 1С'},
 {'link': 'https://superjob.ru/vakansii/programmist-1s-32221172.html',
  'salary': [120000],
  'vacancy': 'Программист 1С'},
 {'link': 'https://superjob.ru/vakansii/inzhener-programmist-scada-sistemy-32352472.html',
  'salary': [0],
  'vacancy': 'Инженер-программист SCADA-системы'},
 {'link': 'https://superjob.ru/vakansii/inzhener-programmist-asutp-32352233.html',
  'salary': [100000],
  'vacancy': 'Инженер-программист АСУТП (ПЛК)'},
 {'link': 'https://superjob.ru/vakansii/programmist-1s-30753446.html',
  'salary': [50000, 200000],
  'vacancy': 'Программист 1С'},
 {'link': 'https://superjob.ru/vakansii/razrabotchik-t-sql-28685862.ht

2) Реализовать сохранение полученных вакансий в СУБД (на выбор SQLite или MongoDB)

In [41]:
from pymongo import MongoClient

client = MongoClient('mongodb://127.0.0.1:27017')
db = client['vacancy_db']
vacancy_db = db.vacancy_db

In [42]:
vacancy_db.insert_many(vacancy_hh)

In [43]:
vacancy_db.insert_many(vacancy_sj)

3) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы.

In [53]:
def find_salary(salary):
    objects = vacancy_db.find({'salary': {'$gt':salary}})
    found_salary = list(objects)
    return found_salary
        
        
pprint(find_salary(250000))

[{'_id': ObjectId('5d654ddb891bf0006490217e'),
  'link': 'https://hh.ru/vacancy/33111410',
  'salary': [150000, 260000],
  'vacancy': 'Backend разработчик (.NET)'},
 {'_id': ObjectId('5d654ddb891bf000649021a8'),
  'link': 'https://hh.ru/vacancy/32983370',
  'salary': [200000, 300000],
  'vacancy': 'Разработчик iOS (Objective-C)'},
 {'_id': ObjectId('5d654ddb891bf000649021cb'),
  'link': 'https://hh.ru/vacancy/32983365',
  'salary': [200000, 300000],
  'vacancy': 'Senior Android Разработчик'}]


4) \* Модифицировать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта